In [ ]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read uploaded files
df= pd.read_csv('Amazon - Movies and TV Ratings.csv')

In [ ]:
df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(4848, 207)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Movie1,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie2,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie3,1.0,2.000000,NaN,2.0,2.00,2.0,2.0,2.0
Movie4,2.0,5.000000,0.000000,5.0,5.00,5.0,5.0,5.0
Movie5,29.0,4.103448,1.496301,1.0,4.00,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...
Movie202,6.0,4.333333,1.632993,1.0,5.00,5.0,5.0,5.0
Movie203,1.0,3.000000,NaN,3.0,3.00,3.0,3.0,3.0
Movie204,8.0,4.375000,1.407886,1.0,4.75,5.0,5.0,5.0
Movie205,35.0,4.628571,0.910259,1.0,5.00,5.0,5.0,5.0


In [ ]:
df.describe().T ['count'].sort_values(ascending=False) [:10]

Movie127    2313.0
Movie140     578.0
Movie16      320.0
Movie103     272.0
Movie29      243.0
Movie91      128.0
Movie92      101.0
Movie89       83.0
Movie158      66.0
Movie108      54.0
Name: count, dtype: float64

# Task1. Which movies have maximum views?

In [ ]:
df.describe().T ['count'].sort_values(ascending=False) [:1].to_frame()

,count
Movie127,2313.0


# Ques1. Which movies have maximum ratings?

In [ ]:
df.drop('user_id', axis=1).sum().sort_values(ascending=False) [:1]

Movie127    9511.0
dtype: float64

# Task 2 - What is the average rating for each movie? Define the top 5 movies with the maximum ratings


In [ ]:
df.drop('user_id', axis=1).mean().sort_values(ascending=False)[:5].to_frame()

,0
Movie1,5.0
Movie66,5.0
Movie76,5.0
Movie75,5.0
Movie74,5.0


# Task3 Define top five movie with least audience

In [ ]:
df.describe().T['count'].sort_values(ascending=True)[:5].to_frame()

,count
Movie1,1.0
Movie71,1.0
Movie145,1.0
Movie69,1.0
Movie68,1.0


# User based Model building


In [ ]:
#importing libiraies for model building
!pip install surprise
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise import SVD
from surprise.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 3.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633994 sha256=ad2cb7a8358e12b10820ea1a2f233b8bdea2e7602df04717847047149e8a305e
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
movie_data = df.melt(id_vars = df.columns[0],value_vars=df.columns[1:],var_name="Movies",value_name="Rating")
movie_data

,user_id,Movies,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


#creating a dataset for training and testing

In [ ]:
rd = Reader(rating_scale=(-1,10))
data = Dataset.load_from_df(movie_data.fillna(0),reader=rd)
data

In [ ]:
train_data,test_data = train_test_split(data,test_size=0.20)


In [ ]:
#Using SVD (Singular Value Descomposition)
svd = SVD()

In [ ]:
svd.fit(train_data)

In [ ]:
pred = svd.test(test_data)

In [ ]:
accuracy.rmse(pred)

RMSE: 0.2794


0.2793754258298331

In [ ]:
accuracy.mae(pred)

MAE:  0.0401


0.04008126472300157

In [ ]:
u_id='AH3QC2PC1VTGP'
mv = 'Movie206'
r_id = 5.0
svd.predict(u_id, mv, r_ui=r_id, verbose= True)

user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 0.01   {'was_impossible': False}


Prediction(uid='AH3QC2PC1VTGP', iid='Movie206', r_ui=5.0, est=0.008270456696576946, details={'was_impossible': False})

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.2793  0.2876  0.2802  0.2824  0.0037  
MAE (testset)     0.0425  0.0434  0.0425  0.0428  0.0004  
Fit time          44.25   46.76   44.38   45.13   1.16    
Test time         4.19    3.81    3.58    3.86    0.25    


{'test_rmse': array([0.2793359 , 0.28763958, 0.28024328]),
 'test_mae': array([0.04245215, 0.04340274, 0.04249328]),
 'fit_time': (44.248100996017456, 46.76270365715027, 44.377259731292725),
 'test_time': (4.186697721481323, 3.8057003021240234, 3.58400297164917)}

In [ ]:
def repeat(ml_type,dframe,min_,max_):
    rd = Reader()
    data = Dataset.load_from_df(dframe,reader=rd)
    print(cross_validate(ml_type, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True))
    print("#"*10)
    u_id = 'AH3QC2PC1VTGP'
    m_id = 'Movie206'
    ra_u = 5.0
    print(ml_type.predict(u_id,mv,r_ui=ra_u,verbose=True))
    print("#"*10)
    print()

In [ ]:
df= df.iloc[:3000, :50]
movie_data = df.melt(id_vars = df.columns[0],value_vars=df.columns[1:],var_name="Movies",value_name="Rating")

In [ ]:
repeat(SVD(),movie_data.fillna(0),-1,10)
repeat(SVD(),movie_data.fillna(movie_data.mean()),-1,10)
repeat(SVD(),movie_data.fillna(movie_data.median()),-1,10)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0323  1.0286  1.0265  1.0291  0.0024  
MAE (testset)     1.0138  1.0124  1.0113  1.0125  0.0010  
Fit time          5.95    5.93    5.92    5.93    0.01    
Test time         0.42    0.34    0.74    0.50    0.17    
{'test_rmse': array([1.03228049, 1.02860601, 1.02652059]), 'test_mae': array([1.01378538, 1.01236525, 1.01131268]), 'fit_time': (5.945601224899292, 5.932776927947998, 5.92068338394165), 'test_time': (0.4231088161468506, 0.3419034481048584, 0.7426743507385254)}
##########
user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 1.00   {'was_impossible': False}
user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 1.00   {'was_impossible': False}
##########



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.0675  0.0463  0.0547  0.0562  0.0087  
MAE (testset)     0.0075  0.0076  0.0073  0.0075  0.0001  
Fit time          5.88    5.91    6.23    6.01    0.16    
Test time         0.37    0.74    0.39    0.50    0.17    
{'test_rmse': array([0.06754646, 0.04633509, 0.05472269]), 'test_mae': array([0.00745746, 0.00759476, 0.00730639]), 'fit_time': (5.87963604927063, 5.907711029052734, 6.234139919281006), 'test_time': (0.3694651126861572, 0.7395086288452148, 0.39429235458374023)}
##########
user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 4.61   {'was_impossible': False}
user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 4.61   {'was_impossible': False}
##########



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.0556  0.0587  0.0714  0.0619  0.0069  
MAE (testset)     0.0052  0.0048  0.0049  0.0049  0.0002  
Fit time          5.88    5.97    5.91    5.92    0.04    
Test time         0.43    0.76    0.33    0.51    0.18    
{'test_rmse': array([0.05558742, 0.05873125, 0.07143595]), 'test_mae': array([0.00516365, 0.00477867, 0.00489044]), 'fit_time': (5.878831624984741, 5.973023176193237, 5.9128313064575195), 'test_time': (0.4277489185333252, 0.7610502243041992, 0.33473801612854004)}
##########
user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 5.00   {'was_impossible': False}
user: AH3QC2PC1VTGP item: Movie206   r_ui = 5.00   est = 5.00   {'was_impossible': False}
##########

